In [1]:
import numpy as np
from os.path import join
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from google.colab import drive

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import classification_report
from sklearn.cluster import MeanShift
import pandas as pd
from google.colab import files

from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten

Using TensorFlow backend.


In [2]:
# mount drive
drive.mount('/content/gdrive', force_remount=True)
root_path = 'gdrive/My Drive/COMP8220-GColab/'

# import directory from gdrive
dataset_directory = '/content/gdrive/My Drive/COMP8220 - Datasets/Project/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# import datasets
images_train = np.load(join(dataset_directory, 'train_images.npy'))
labels_train = np.load(join(dataset_directory, 'train_labels.npy')) # [black, blond, brown, gray]

images_val = np.load(join(dataset_directory, 'val_images.npy'))
labels_val = np.load(join(dataset_directory, 'val_labels.npy'))
files_val = np.load(join(dataset_directory, 'val_files.npy'))

images_test_public = np.load(join(dataset_directory, 'test_public_images.npy'))
files_test_public = np.load(join(dataset_directory, 'test_public_files.npy'))

In [0]:
image_size = 48
hair_colours = ['black', 'blond', 'brown', 'gray']

# reshape and scale values in [0,1] interval 
images_train = images_train.reshape([-1, image_size, image_size, 3]) / 255.0
images_val = images_val.reshape([-1, image_size, image_size, 3]) / 255.0
images_test_public = images_test_public.reshape([-1, image_size, image_size, 3]) / 255.0

**Conventional Machine Learning Methods**

In [55]:
# transform from 4D to 2D
vol_shape = images_train.shape[1:4]
n_vol =  np.prod(vol_shape)
images_train_2d = images_train.reshape(images_train.shape[0], n_vol)
images_val_2d = images_val.reshape(images_val.shape[0], n_vol)
images_test_public_2d = images_test_public.reshape(images_test_public.shape[0], n_vol)

print(images_train_2d.shape)
print(images_val_2d.shape)
print(images_test_public_2d.shape)

(98479, 6912)
(12270, 6912)
(11921, 6912)


In [56]:
## SVM train
svm_clf = BaggingClassifier(base_estimator = SVC(kernel = 'linear', random_state = 42, C = 10.0), n_estimators = 10, max_samples = 1000, random_state = 42)
svm_clf.fit(images_train_2d, labels_train)

ValueError: ignored

In [0]:
## SVM val predict
svm_predictions = svm_clf.predict(images_val_g2)

In [0]:
print(classification_report(labels_val, svm_predictions, target_names=hair_colours))

              precision    recall  f1-score   support

       black       0.61      0.81      0.70      4007
       blond       0.72      0.83      0.77      2852
       brown       0.69      0.48      0.57      4507
        gray       0.78      0.43      0.55       904

    accuracy                           0.67     12270
   macro avg       0.70      0.64      0.65     12270
weighted avg       0.68      0.67      0.66     12270



In [0]:
## RF train
rf_clf = BaggingClassifier(base_estimator=RandomForestClassifier(random_state=42), n_estimators=10, max_samples=1000, random_state=42)
rf_clf.fit(images_train_g2, labels_train)

BaggingClassifier(base_estimator=RandomForestClassifier(bootstrap=True,
                                                        ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='gini',
                                                        max_depth=None,
                                                        max_features='auto',
                                                        max_leaf_nodes=None,
                                                        max_samples=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
     

In [0]:
## RF val predict
rf_predictions = rf_clf.predict(images_val_g2)

In [0]:
print(classification_report(labels_val, rf_predictions, target_names=hair_colours))

              precision    recall  f1-score   support

       black       0.70      0.83      0.76      4007
       blond       0.71      0.83      0.77      2852
       brown       0.70      0.62      0.66      4507
        gray       0.83      0.22      0.35       904

    accuracy                           0.71     12270
   macro avg       0.74      0.63      0.63     12270
weighted avg       0.71      0.71      0.69     12270



In [0]:
## SGD train
#sgd_clf = BaggingClassifier(base_estimator=SGDClassifier(loss="hinge", penalty="l2", max_iter=1000), n_estimators=10, max_samples=1000, random_state=42)
sgd_clf = SGDClassifier(loss = 'log')
sgd_clf.fit(images_train_2d, labels_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1000,
              n_iter_no_change=5, n_jobs=None, penalty='l2', power_t=0.5,
              random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [0]:
sgd_predictions = sgd_clf.predict(images_val_2d)

In [0]:
print(classification_report(labels_val, sgd_predictions, target_names=hair_colours))

              precision    recall  f1-score   support

       black       0.68      0.93      0.79      4007
       blond       0.80      0.88      0.84      2852
       brown       0.85      0.49      0.62      4507
        gray       0.69      0.83      0.76       904

    accuracy                           0.75     12270
   macro avg       0.76      0.78      0.75     12270
weighted avg       0.77      0.75      0.74     12270



**Deep Learning ( CNN )**

In [0]:
# convert labels to categories
labels_train = to_categorical(labels_train)
labels_val = to_categorical(labels_val)

In [0]:
## Sequential Model CNN
model = Sequential()
model.add(Conv2D(128, kernel_size=4, activation='relu', input_shape=(48,48,3)))
model.add(Conv2D(64, kernel_size=4, activation='relu'))
model.add(Conv2D(64, kernel_size=4, activation='relu'))
model.add(Flatten())
model.add(Dense(4, activation='softmax'))

In [0]:
#compile model using accuracy to measure model performance
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
#train the model
model.fit(images_train, labels_train, validation_data=(images_val, labels_val), epochs=4)

Train on 98479 samples, validate on 12270 samples
Epoch 1/4
98479/98479 [==============================] - 2844s 29ms/step - loss: 0.3975 - accuracy: 0.8499 - val_loss: 0.3327 - val_accuracy: 0.8717
Epoch 2/4
98479/98479 [==============================] - 2775s 28ms/step - loss: 0.3006 - accuracy: 0.8888 - val_loss: 0.2927 - val_accuracy: 0.8859
Epoch 3/4
98479/98479 [==============================] - 2746s 28ms/step - loss: 0.2631 - accuracy: 0.9024 - val_loss: 0.3051 - val_accuracy: 0.8825
Epoch 4/4
98479/98479 [==============================] - 2757s 28ms/step - loss: 0.2326 - accuracy: 0.9151 - val_loss: 0.2921 - val_accuracy: 0.8929


In [0]:
cnn_pred = model.predict(images_test_public)

In [0]:
df_cnn = pd.DataFrame(data = cnn_pred[0:])
pred = df_cnn.idxmax(axis=1)

In [0]:
## create Dataframe
df = pd.DataFrame(columns = ['ID', 'Prediction'])
df['ID'] = files_test_public
df['Prediction'] = pred

In [0]:
## export test result
df.to_csv('45799857_04.csv',header=True, index=False)
!cp 45799857_04.csv dataset_directory